In [1]:
#KERAS_BACKEND="theano"
import numpy as np
import word2vec_utils as w2v
import data
from data_utils import split_dataset 
import keras
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json
import subprocess
import shlex
import tkinter as tk
try:
    import ttk as ttk
    import ScrolledText
except ImportError:
    import tkinter.ttk as ttk
    import tkinter.scrolledtext as ScrolledText
import time
class neural_chatbot():
    
    def __init__(self):
        #self.initialize()
        self.load_model()
        self.w2v_model = w2v.initialize()
        self.count=0
        self.out='1 9 1 @Hi'
        print( 'Loaded word_frequencies data from disk' )
        self.word_freqs = np.load('words_in_order_of_freq.npy') 
        self.embed_dim = 101
        self.data_dim = 1
        self.timesteps = 26
        self.vocab_size = 10000
        self.emot_size = 4
        self.max_sent_length = 30
        self.reinforce = 'neutral'
    def save_model(self):
        # serialize model to JSON
        model_json = self.one_hot_chat_model.to_json()
        with open("one_hot_chat_model.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        self.one_hot_chat_model.save_weights("one_hot_chat_net.h5")
        print("Saved model to disk")
    
    def load_model(self):
        # load json and create model
        json_file = open('one_hot_chat_model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.one_hot_chat_model = model_from_json(loaded_model_json)
        # load weights into new model
        self.one_hot_chat_model.load_weights("one_hot_chat_net.h5")
        print("Loaded model from disk")
    
    def RateSentiment(self,sentiString):
        #open a subprocess using shlex to get the command line string into the correct args list format
        p = subprocess.Popen(shlex.split("C:/ProgramData/Oracle/java/javapath/java -jar C:/Users/anul/Downloads/SentiStrength.jar stdin sentidata C:/Users/anul/Downloads/SentStrength_Data_Sept2011/"),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE,universal_newlines=True)
        #communicate via stdin the string to be rated. Note that all spaces are replaced with +
        stdout_text, stderr_text = p.communicate(sentiString.replace(" ","+"))
        #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1-5
        stdout_text = stdout_text.rstrip().replace("\t","")
        return stdout_text   
    
            
    def reinforcement_learn(self,user_emotion,sys_response):
        total_emo = int(user_emotion[0]) - int(user_emotion[3])
        sys_res = np.argmax(sys_response,axis=2)
        adam = keras.optimizers.Adam(lr = 100)#default 0.001
        self.one_hot_chat_model.compile( optimizer=adam,loss = 'categorical_crossentropy',metrics = ['accuracy'])
        if total_emo > 0:
            #sys_res = (10*sys_res)+10
            sys_res_categorical=to_categorical(sys_res.astype(int),self.vocab_size+3).reshape((1,self.max_sent_length,self.vocab_size+3))
            self.reinforce='positive'
            self.one_hot_chat_model.fit(self.user_input,sys_res_categorical,epochs=3)
        else:
            sys_res_categorical=to_categorical(sys_res.astype(int),self.vocab_size+3).reshape((1,self.max_sent_length,self.vocab_size+3))*(-1)
            self.reinforce='negative'
            self.one_hot_chat_model.fit(self.user_input,sys_res_categorical,epochs=3)
        self.save_model()
        self.load_model()
        
    #def convert3d_to_4d(sys_response3d):
    #    sys_res3d_standardized = (10*sys_res)+10
    #    sys_response = to_categorical(sys_res3d_standardized.astype(int),20).reshape((1,300,-1,20))

    def model_response(self,User_B):
        #User_A1 = "5 5 1 @I really love you. I think I want to marry you!"
        #User_B = "1 5 5 @yeah well I hate you and never want to see you again! "
        #User_A1 = w2v.vectorize( w2v_model, User_A1 , pad_length = max_sent_length)
        Sen_User_B = self.RateSentiment(User_B)
        print(Sen_User_B)
        self.count=self.count+1
        if self.count>1:
            self.reinforcement_learn(Sen_User_B,self.sys_prediction)
        
        User_B = Sen_User_B[0]+" "+str(11-int(Sen_User_B[0])-int(Sen_User_B[3]))+" "+Sen_User_B[3]+" "+"@"+User_B
        self.User_B_in  = w2v.vectorize( User_B, pad_length = 30,model=self.w2v_model ) 
        User_A = w2v.vectorize( self.out, pad_length = 30,model=self.w2v_model ) 
        self.user_input = [ np.array( [User_A]).reshape(1,30,101), np.array([self.User_B_in]).reshape(1,30,101) ]
        #print(self.sys_prediction.shape)
        self.sys_prediction = self.one_hot_chat_model.predict(self.user_input)
        print(self.sys_prediction.shape)
        sentence=np.argmax(self.sys_prediction,axis=2)
        #w2v.one_hot_unvectorize(np.argmax(self.sys_prediction,axis=2))
        pred_words=[]
        for i in range(30):
            pred_words.append(self.word_freqs[sentence[0,i]].decode('utf-8'))
        self.out=" ".join(pred_words)
        print(self.out)    
        return (self.out)
        #sys_pred = np.argmax(prediction, axis=2)
        #self.sys_prediction = (sys_pred-10)/10
        #return self.most_likely_sent( self.sys_prediction[0], metadata['w2idx'].keys(),self.w2v_model)
    
class TkinterGUIExample(tk.Tk):

    def __init__(self, *args, **kwargs):
        """
        Create & set window variables.
        """
        tk.Tk.__init__(self, *args, **kwargs)

        self.title("Di-Feelbot")
        self.neuralbot = neural_chatbot()
        self.initialize()

    def initialize(self):
        """
        Set window layout.
        """
        self.grid()

        self.respond = ttk.Button(self, text='Get Response', command=self.get_response)
        self.respond.grid(column=0, row=0, sticky='nesw', padx=3, pady=3)

        self.usr_input = ttk.Entry(self, state='normal')
        self.usr_input.grid(column=1, row=0, sticky='nesw', padx=3, pady=3)

        self.conversation_lbl = ttk.Label(self, anchor=tk.E, text='Conversation:')
        self.conversation_lbl.grid(column=0, row=5, sticky='nesw', padx=3, pady=3)

        self.conversation = ScrolledText.ScrolledText(self, state='disabled')
        self.conversation.grid(column=0, row=6, columnspan=2, sticky='nesw', padx=3, pady=3)
        
        self.emo_lbl = ttk.Label(self, text='Sys emotion')
        self.emo_lbl.grid(column=0, row=1, sticky='nesw', padx=3, pady=3)
        
        self.emoPosvalue_lbl = ttk.Label(self, text='Pos emotion',foreground='blue')
        self.emoPosvalue_lbl.grid(column=0, row=2, sticky='nesw', padx=3, pady=3)
        
        self.emoNegvalue_lbl = ttk.Label(self, text='Neg emotion',foreground='red')
        self.emoNegvalue_lbl.grid(column=0, row=3, sticky='nesw', padx=3, pady=3)
        
        self.emoNeuvalue_lbl = ttk.Label(self, text='Neu emotion',foreground='black')
        self.emoNeuvalue_lbl.grid(column=0, row=4, sticky='nesw', padx=3, pady=3)
        
        self.uemo_lbl = ttk.Label(self, text='User emotion')
        self.uemo_lbl.grid(column=1, row=1, sticky='nesw', padx=3, pady=3)
        
        self.uemoPosvalue_lbl = ttk.Label(self, text='Pos emotion',foreground='blue')
        self.uemoPosvalue_lbl.grid(column=1, row=2, sticky='nesw', padx=3, pady=3)
        
        self.uemoNegvalue_lbl = ttk.Label(self, text='Neg emotion',foreground='red')
        self.uemoNegvalue_lbl.grid(column=1, row=3, sticky='nesw', padx=3, pady=3)
        
        self.uemoNeuvalue_lbl = ttk.Label(self, text='Neu emotion',foreground='black')
        self.uemoNeuvalue_lbl.grid(column=1, row=4, sticky='nesw', padx=3, pady=3)
        
        self.reinforce_lbl = ttk.Label(self, text='Reinforcement',foreground='red')
        self.reinforce_lbl.grid(column=2, row=1, sticky='nesw', padx=3, pady=3)
    
        
    def get_response(self):
        """
        Get a response from the chatbot and display it.
        """
        user_input = self.usr_input.get()
        self.usr_input.delete(0, tk.END)

        response = self.neuralbot.model_response(user_input)
        x=self.neuralbot.RateSentiment(user_input)
        self.emoPosvalue_lbl['text'] = response[0]
        self.emoNegvalue_lbl['text'] = response[4]
        self.emoNeuvalue_lbl['text'] = response[2]
        self.uemoPosvalue_lbl['text'] = x[0]
        self.uemoNegvalue_lbl['text'] = x[3]
        self.uemoNeuvalue_lbl['text'] = str(11-int(x[0])-int(x[3]))
        self.reinforce_lbl['text'] = self.neuralbot.reinforce + ' Reinforcement'
        self.conversation['state'] = 'normal'
        self.conversation.insert(
            tk.END, "Human: " + user_input + "\n" + "ChatBot: " + str(response[7:]) + "\n"
        )
        self.conversation['state'] = 'disabled'

        time.sleep(0.5)


gui_example = TkinterGUIExample()
gui_example.mainloop()

C:\Users\anul\Anaconda2\envs\python35\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Loaded model from disk
Loaded word_frequencies data from disk
1 -1
(1, 30, 10003)
poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom
1 -2
Epoch 1/3
1/1 [==============================] - 11s - loss: -1.1921e-07 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s - loss: -1.1921e-07 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s - loss: -1.1921e-07 - acc: 0.0000e+00
Saved model to disk
Loaded model from disk
(1, 30, 10003)
poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom poolroom


In [1]:
## Credit to http://sentistrength.wlv.ac.uk/documentation/Python.txt for the code
 
import shlex, subprocess
 
def RateSentiment(sentiString):
    #open a subprocess using shlex to get the command line string into the correct args list format
    p = subprocess.Popen(shlex.split("C:/ProgramData/Oracle/java/javapath/java -jar C:/Users/anul/Downloads/SentiStrength.jar stdin sentidata \
                                     C:/Users/anul/Downloads/Sentstrength_Data_Sept2011/ "),
                         stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE,universal_newlines=True)
    #communicate via stdin the string to be rated. Note that all spaces are replaced with +
    stdout_text, stderr_text = p.communicate(sentiString.replace(" ","+"))
    #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1-5
    stdout_text = stdout_text.rstrip().replace("\t","")
     

    return stdout_text
print(RateSentiment("I hate you"))

1 -4


In [17]:
p=RateSentiment('hate')
print(p)

ValueError: No closing quotation

In [23]:
from subprocess import *

def jarWrapper(*args):
    process = Popen(['java', '-jar']+list(args), stdout=PIPE, stderr=PIPE)
    ret = []
    while process.poll() is None:
        line = process.stdout.readline()
        if line != '' and line.endswith('\n'):
            ret.append(line[:-1])
    stdout, stderr = process.communicate()
    ret += stdout.split('\n')
    if stderr != '':
        ret += stderr.split('\n')
    ret.remove('')
    return ret

args = ['C:/Users/anul/Downloads/SentiStrength.jar', 'text', 'hate','sentidata', 'C:/Users/anul/Downloads/SentStrength_Data_Sept2011/'] # Any number of args to be passed to the jar file

result = jarWrapper(*args)

print(result)

OSError: [WinError 6] The handle is invalid

In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Merge
#from keras.utils.visualize_util import plot, to_graph
#import theano.d3viz as d3v
import word2vec_utils_1 as w2v
import data
import numpy as np
from data_utils import split_dataset 

embed_dim = 300
data_dim = 1
timesteps = 26
vocab_size = 8100
emot_size = 4
max_sent_length = timesteps + emot_size

model_A = Sequential()
model_A.add(LSTM(embed_dim,input_shape=(max_sent_length,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))

model_A.compile(loss='cosine_proximity',
      optimizer='rmsprop',
      metrics=['accuracy']
      )
model_A.load_weights('model_A_weights.h5')

model_B = Sequential()
model_B.add(LSTM(embed_dim,input_shape=(max_sent_length,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))

model_B.compile(loss='cosine_proximity',
      optimizer='rmsprop',
      metrics=['accuracy']
      )
model_B.load_weights('model_B_weights.h5')

model_ABA = Sequential()
model_ABA.add(Merge([model_A, model_B], mode = 'concat'))
model_ABA.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model_ABA.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model_ABA.add(TimeDistributed(Dense(vocab_size,activation='softmax')))

model_ABA.compile(loss='cosine_proximity',
      optimizer='rmsprop',
      metrics=['accuracy']
      )
model_ABA.load_weights('model_ABA_weights.h5')


def cosign_similarity( a, b):
    dot = np.dot(a,b)
    norm = np.linalg.norm(a-b)
    return( dot/norm )

def most_likely_word( prediction, words, w2v_model ):
    ret = ""
    max_so_far = cosign_similarity( np.array([1]+[0]*299),prediction ) #must be better than blank
    #Best word in dictionary
    for word in words:
        similarity = cosign_similarity( w2v.vectorize(w2v_model,word)[0], prediction )
        if similarity > max_so_far:
            max_so_far = similarity
            ret = word    
    return( ret )

def most_likely_sent( prediction_vec, words, w2v_model ):
    predicted_words = []
    for prediction in prediction_vec:
        predicted_words.append( most_likely_word(prediction,words,w2v_model))
    ret = " ".join(predicted_words)
    return( ret )



Using TensorFlow backend.
C:\Users\anul\Anaconda2\envs\python35\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\anul\Anaconda2\envs\python35\lib\site-packages\ipykernel\__main__.py:20: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(30, 300))`
C:\Users\anul\Anaconda2\envs\python35\lib\site-packages\ipykernel\__main__.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(30, 300))`
C:\Users\anul\Anaconda2\envs\python35\lib\site-packages\ipykernel\__main__.py:38: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\anul\Anaconda2\envs\python35\lib\site-pa

In [4]:
model_A.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 30, 300)           721200    
Total params: 721,200
Trainable params: 721,200
Non-trainable params: 0
_________________________________________________________________


In [2]:
w2v_model = w2v.initialize()

In [3]:
w2v.save_unknown_vectors({})

In [4]:
w2v.get_unknown_vectors()

{}

In [5]:
w2v.unvectorize_initialize()

In [3]:
#unknown_vectors=[]
metadata, filtered_A1, filtered_B, filtered_A2 = data.load_data()
User_A1 = "5 5 1 @I really love you. I think I want to marry you!"
User_B = "1 5 5 @yeah well I hate you and never want to see you again! "
User_A1 = w2v.vectorize( sentence=User_A1,model=w2v_model , pad_length = max_sent_length) 
User_B  = w2v.vectorize( sentence=User_B,model=w2v_model,  pad_length = max_sent_length)
print(np.array([User_B[0]]).shape)
user_input = [ np.array( [User_A1]), np.array([User_B]) ]
prediction = model_ABA.predict(user_input)
print(most_likely_sent( prediction[0], metadata['w2idx'].keys(),w2v_model))

UnpicklingError: the STRING opcode argument must be quoted

In [3]:
from keras.models import model_from_json
model_json = model_ABA.to_json()
with open("model_ABA.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_ABA.save_weights("model_ABA.h5")
print("Saved model to disk")

Saved model to disk


In [2]:
import subprocess
import shlex
import Tkinter as tk
try:
    import ttk as ttk
    import ScrolledText
except ImportError:
    import tkinter.ttk as ttk
    import tkinter.scrolledtext as ScrolledText
class TkinterGUIExample(tk.Tk):

    def __init__(self, *args, **kwargs):
        """
        Create & set window variables.
        """
        tk.Tk.__init__(self, *args, **kwargs)

        self.title("Di-Feelbot")
        #self.neuralbot = neural_chatbot()
        self.initialize()

    def initialize(self):
        """
        Set window layout.
        """
        self.grid()

        self.respond = ttk.Button(self, text='Get Response', command=self.get_response)
        self.respond.grid(column=0, row=0, sticky='nesw', padx=3, pady=3)

        self.usr_input = ttk.Entry(self, state='normal')
        self.usr_input.grid(column=1, row=0, sticky='nesw', padx=3, pady=3)

        self.conversation_lbl = ttk.Label(self, anchor=tk.E, text='Conversation:')
        self.conversation_lbl.grid(column=0, row=5, sticky='nesw', padx=3, pady=3)

        self.conversation = ScrolledText.ScrolledText(self, state='disabled')
        self.conversation.grid(column=0, row=6, columnspan=2, sticky='nesw', padx=3, pady=3)
        
        self.emo_lbl = ttk.Label(self, text='Sys emotion')
        self.emo_lbl.grid(column=0, row=1, sticky='nesw', padx=3, pady=3)
        
        self.emoPosvalue_lbl = ttk.Label(self, text='Pos emotion',foreground='blue')
        self.emoPosvalue_lbl.grid(column=0, row=2, sticky='nesw', padx=3, pady=3)
        
        self.emoNegvalue_lbl = ttk.Label(self, text='Neg emotion',foreground='red')
        self.emoNegvalue_lbl.grid(column=0, row=3, sticky='nesw', padx=3, pady=3)
        
        self.emoNeuvalue_lbl = ttk.Label(self, text='Neu emotion',foreground='black')
        self.emoNeuvalue_lbl.grid(column=0, row=4, sticky='nesw', padx=3, pady=3)
        
        self.uemo_lbl = ttk.Label(self, text='User emotion')
        self.uemo_lbl.grid(column=1, row=1, sticky='nesw', padx=3, pady=3)
        
        self.uemoPosvalue_lbl = ttk.Label(self, text='Pos emotion',foreground='blue')
        self.uemoPosvalue_lbl.grid(column=1, row=2, sticky='nesw', padx=3, pady=3)
        
        self.uemoNegvalue_lbl = ttk.Label(self, text='Neg emotion',foreground='red')
        self.uemoNegvalue_lbl.grid(column=1, row=3, sticky='nesw', padx=3, pady=3)
        
        self.uemoNeuvalue_lbl = ttk.Label(self, text='Neu emotion',foreground='black')
        self.uemoNeuvalue_lbl.grid(column=1, row=4, sticky='nesw', padx=3, pady=3)
        
    def RateSentiment(self,sentiString):
        #open a subprocess using shlex to get the command line string into the correct args list format
        p = subprocess.Popen(shlex.split("C:/ProgramData/Oracle/java/javapath/java -jar C:/Users/anul/Downloads/SentiStrength.jar stdin sentidata C:/Users/anul/Downloads/SentStrength_Data_Sept2011/"),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        #communicate via stdin the string to be rated. Note that all spaces are replaced with +
        stdout_text, stderr_text = p.communicate(sentiString.replace(" ","+"))
        #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1-5
        stdout_text = stdout_text.rstrip().replace("\t","")
        return stdout_text
        
    def get_response(self):
        """
        Get a response from the chatbot and display it.
        """
        user_input = self.usr_input.get()
        self.usr_input.delete(0, tk.END)

        response = '1 9 1 @Hi how are you'#self.neuralbot.model_response(user_input)
        x=self.RateSentiment(user_input)
        self.emoPosvalue_lbl['text'] = response[0]
        self.emoNegvalue_lbl['text'] = response[4]
        self.emoNeuvalue_lbl['text'] = response[2]
        self.uemoPosvalue_lbl['text'] = x[0]
        self.uemoNegvalue_lbl['text'] = x[3]
        self.uemoNeuvalue_lbl['text'] = str(11-int(x[0])-int(x[3]))
        self.conversation['state'] = 'normal'
        self.conversation.insert(
            tk.END, "Human: " + user_input + "\n" + "ChatBot: " + str(response[7:]) + "\n"
        )
        self.conversation['state'] = 'disabled'

        time.sleep(0.5)


gui_example = TkinterGUIExample()
gui_example.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\anul\Anaconda2\envs\python2\lib\lib-tk\Tkinter.py", line 1542, in __call__
    return self.func(*args)
  File "<ipython-input-2-de1a6c221b0b>", line 94, in get_response
    time.sleep(0.5)
NameError: global name 'time' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\anul\Anaconda2\envs\python2\lib\lib-tk\Tkinter.py", line 1542, in __call__
    return self.func(*args)
  File "<ipython-input-2-de1a6c221b0b>", line 94, in get_response
    time.sleep(0.5)
NameError: global name 'time' is not defined


In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, concatenate, Input
from keras.layers.wrappers import Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
import word2vec_utils as w2v
import data
import numpy as np
import re
import random
from data_utils import split_dataset 
from chat_constants import *
from keras.utils.np_utils import to_categorical
LSTM_DROPOUT = 0.15
A1_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

A1_layer1 = Bidirectional( LSTM(EMBED_DIM,name = "A1_layer1", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT) )
A1_layer2 = LSTM(EMBED_DIM*2,name = "A1_layer2", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A1_net = A1_layer2(A1_layer1(A1_input))

B_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

B_layer1 = Bidirectional( LSTM(EMBED_DIM,name = "B_layer1", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT) )
B_layer2 = LSTM(EMBED_DIM*2,name = "B_layer2", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

B_net = B_layer2(B_layer1(B_input))

combined = concatenate([A1_net,B_net])
A2_pred1 = LSTM(EMBED_DIM*4, name = "A2_layer1", return_sequences = True, dropout = LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A2_pred2 = TimeDistributed(Dense(VOCAB_SIZE + 3, name = "A2_layer2", activation = 'softmax' ) )

A2_net =A2_pred2(A2_pred1(combined))

one_hot_chat_model = Model(inputs = [ A1_input,B_input], outputs = [A2_net])
adam = keras.optimizers.Adam(lr = 0.01)#default 0.001
one_hot_chat_model.compile( optimizer=adam,loss = 'categorical_crossentropy',metrics = ['accuracy'])

one_hot_chat_model.summary()
one_hot_chat_model.load_weights("one_hot_chat_net.h5")
print("Loaded model from disk")


Using TensorFlow backend.
C:\Users\anul\Anaconda2\envs\python35\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30, 101)       0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 30, 101)       0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 30, 202)       164024      input_1[0][0]                    
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 30, 202)       164024      input_2[0][0]                    
___________________________________________________________________________________________

In [2]:
model_json = one_hot_chat_model.to_json()
with open("one_hot_chat_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
one_hot_chat_model.save_weights("one_hot_chat_net.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
word_freqs = np.load('words_in_order_of_freq.npy') 
print(word_freqs.decode('utf-8'))

AttributeError: 'numpy.ndarray' object has no attribute 'decode'